In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, sum as _sum, when, lit
from pyspark.sql.functions import rand, when

# Importando bases
user_orders = spark.table('workspace.tabelas_ifood.user_orders').drop('price_range')
user_groups = spark.table('workspace.tabelas_ifood.user_labels')
user_orders = user_orders.join(user_groups,on='customer_id',how='left')

margem = 0.25
custo_cupom = 10
roi_target = 0.1

In [0]:
# Testando segmentacao de frequencia - usuarios com pouca atividade

frequency_group = user_orders.filter(user_orders['frequency_range'] == 'baixa')

frequency_group_metrics = frequency_group.groupBy('is_target').agg(
    count('customer_id').alias('usuarios'),
    avg('order_total_amount').alias('ticket_medio'),
    _sum('order_total_amount').alias('valor_total'),
    count('order_id').alias('num_pedidos')
)

In [0]:
# Impacto incremental e ROI

control = frequency_group_metrics.filter(frequency_group_metrics['is_target'] == 'control').collect()[0]
target = frequency_group_metrics.filter(frequency_group_metrics['is_target'] == 'target').collect()[0]

# Cálculo incremental
pedidos_extra = target["num_pedidos"] - control["num_pedidos"]
valor_extra = target["valor_total"] - control["valor_total"]
lucro_extra = valor_extra * margem
custo_total_cupons = target["num_pedidos"] * custo_cupom
roi = (lucro_extra - custo_total_cupons) / custo_total_cupons if custo_total_cupons > 0 else None

# Resultado
print("=== Resultado da Campanha ===")
print(f"Pedidos extras (incrementais): {pedidos_extra}")
print(f"Valor extra movimentado: R$ {valor_extra:,.2f}")
print(f"Lucro estimado extra (margem 25%): R$ {lucro_extra:,.2f}")
print(f"Custo total dos cupons: R$ {custo_total_cupons:,.2f}")
print(f"ROI da campanha: {roi:.2f}" if roi is not None else "ROI: indefinido (divisão por zero)")

=== Viabilidade Financeira da Campanha ===
Pedidos extras (incrementais): 146127
Valor extra movimentado: R$ 7,056,370.93
Lucro estimado extra (margem 25%): R$ 1,764,092.73
Custo total dos cupons: R$ 5,169,920.00
ROI da campanha: -0.66


In [0]:
lucro_target = (roi_target*custo_total_cupons) + custo_total_cupons
lucro_gap = lucro_target - lucro_extra
total_pedidos = lucro_gap/margem
numero_pedidos = total_pedidos/target['ticket_medio']
taxa_retencao_target = numero_pedidos/target['usuarios']

# Expectativa de retorno
print("=== Meta de retenção ===")
print(f"Meta de lucro: R$ {lucro_gap:,.2f}")
print(f"Valor total de pedidos: R$ {total_pedidos:,.2f}")
print(f"Número de pedidos: {numero_pedidos:,.2f}")
print(f"Taxa de retenção necessária: {taxa_retencao_target:,.2f}")

=== Meta de retenção ===
Meta de lucro: R$ 3,922,819.27
Valor total de pedidos: R$ 15,691,277.07
Número de pedidos: 326,661.80
Taxa de retenção necessária: 0.63


In [0]:
# Testando segmentacao de ticket medio - usuarios com ticket medio alto

price_group = user_orders.filter(user_orders['price_range'] == '$$$$')

price_group_metrics = price_group.groupBy('is_target').agg(
    count('customer_id').alias('usuarios'),
    avg('order_total_amount').alias('ticket_medio'),
    _sum('order_total_amount').alias('valor_total'),
    count('order_id').alias('num_pedidos')
)

In [0]:
# Impacto incremental e ROI

control = price_group_metrics.filter(price_group_metrics['is_target'] == 'control').collect()[0]
target = price_group_metrics.filter(price_group_metrics['is_target'] == 'target').collect()[0]

# Cálculo incremental
pedidos_extra = target["num_pedidos"] - control["num_pedidos"]
valor_extra = target["valor_total"] - control["valor_total"]
lucro_extra = valor_extra * margem
custo_total_cupons = target["num_pedidos"] * custo_cupom
roi = (lucro_extra - custo_total_cupons) / custo_total_cupons if custo_total_cupons > 0 else None

# Resultado
print("=== Resultado da Campanha ===")
print(f"Pedidos extras (incrementais): {pedidos_extra}")
print(f"Valor extra movimentado: R$ {valor_extra:,.2f}")
print(f"Lucro estimado extra (margem 25%): R$ {lucro_extra:,.2f}")
print(f"Custo total dos cupons: R$ {custo_total_cupons:,.2f}")
print(f"ROI da campanha: {roi:.2f}" if roi is not None else "ROI: indefinido (divisão por zero)")

=== Resultado da Campanha ===
Pedidos extras (incrementais): 151432
Valor extra movimentado: R$ 10,945,111.68
Lucro estimado extra (margem 25%): R$ 2,736,277.92
Custo total dos cupons: R$ 5,272,670.00
ROI da campanha: -0.48


In [0]:
lucro_target = (roi_target*custo_total_cupons) + custo_total_cupons
lucro_gap = lucro_target - lucro_extra
total_pedidos = lucro_gap/margem
numero_pedidos = total_pedidos/target['ticket_medio']
taxa_retencao_target = numero_pedidos/target['usuarios']

# Expectativa de retorno
print("=== Meta de retenção ===")
print(f"Meta de lucro: R$ {lucro_gap:,.2f}")
print(f"Valor total de pedidos: R$ {total_pedidos:,.2f}")
print(f"Número de pedidos: {numero_pedidos:,.2f}")
print(f"Taxa de retenção necessária: {taxa_retencao_target:,.2f}")

=== Meta de retenção ===
Meta de lucro: R$ 3,063,659.08
Valor total de pedidos: R$ 12,254,636.32
Número de pedidos: 163,068.17
Taxa de retenção necessária: 0.31
